In [1]:
#  Hello! Thanks for checking this out.
#  This is a start into using some basic stats to predict the outcome of a future race.
#  Unfortunately F1 is fairly constant, so there isn't a ton of room to improve upon
#  the season results to date and grid position. But if nothing else we can blend those.
#  So here is a way to get some rolling calculations and start predicting future races.
#
#  I started this work with the same data source to show my nine year old son simple 
#  linear blends with a data subject we are both learning and enjoying during lockdown: racing
#
#  So it's not impossible I'll enhance this as the 2020 season progresses.
#  Maybe even put a GBM on top of it eventually.
#  Cheers!

In [2]:
library(data.table)
library(Metrics)

drivers<-fread("../input/formula-1-world-championship-1950-2020/drivers.csv")
races<-fread("../input/formula-1-world-championship-1950-2020/races.csv")
quali<-fread("../input/formula-1-world-championship-1950-2020/qualifying.csv")
results<-fread("../input/formula-1-world-championship-1950-2020/results.csv")

constructors<-fread("../input/formula-1-world-championship-1950-2020/constructors.csv")
### I'm going to smooth a few constructor histories so that when we go to calculate
###  rolling team points, we have a smooth transition.
###  So far I've only done it with the two that affect current drivers the most, but
###  when continuity is reasonable, it is probably best to do this for former teams
###  and future teams (Aston/Racing Point?)
constructors[,mergedId:=constructorId]  ## set up mergedId default to be a copy of constructorId
constructors[constructorId==10,mergedId:=211] ## Force India / Racing Point
constructors[constructorId==15,mergedId:=51] ## Sauber / Alfa Romeo

In [3]:
#### ADD UPDATED RACE DATA IN KERNEL
#### We are one race behind in the main data set--2020 Hungarian Grand Prix. 
#### I could create some CSVs to catch up, but for now, I'm just going to 
#### add the data direclty in the noteook. I'll try and clean
####  up these notebooks in the future, including adding proper markdown.
#### But this way, it's all in one place with one source, instead of two.

races[year==2020][order(date)]
#### races we need are 1043 (Hungarian Grand Prix) and then prep 1044 (British)
#### this kernel does not require the full results: 
####  raceId, driverId, constructorId, grid, pos, positionText, qualifying
####  and the basis for the driver/constructor pairs can be found from Styrian
hungarian<-results[raceId==1042
                   ,.(raceId=1043,resultId=.I+100000,driverId,constructorId
                      ,number,grid=0,position=0,positionText="",positionOrder=0)]
hungarian[driverId==1,`:=`(grid=1,position=1,positionText="1",positionOrder=1)] ## hamilton
hungarian[driverId==822,`:=`(grid=2,position=3,positionText="3",positionOrder=3)] ## bottas
hungarian[driverId==830,`:=`(grid=7,position=2,positionText="2",positionOrder=2)] ## max_verstappen
hungarian[driverId==848,`:=`(grid=13,position=5,positionText="5",positionOrder=5)] ## albon
hungarian[driverId==846,`:=`(grid=8,position=13,positionText="13",positionOrder=13)] ## norris
hungarian[driverId==815,`:=`(grid=4,position=7,positionText="7",positionOrder=7)] ## perez
hungarian[driverId==840,`:=`(grid=3,position=4,positionText="4",positionOrder=4)] ## stroll
hungarian[driverId==817,`:=`(grid=11,position=8,positionText="8",positionOrder=8)] ## ricciardo
hungarian[driverId==832,`:=`(grid=9,position=9,positionText="9",positionOrder=9)] ## sainz
hungarian[driverId==826,`:=`(grid=17,position=12,positionText="12",positionOrder=12)] ## kvyat
hungarian[driverId==8,`:=`(grid=20,position=15,positionText="15",positionOrder=15)] ## raikkonen
hungarian[driverId==825,`:=`(grid=16,position=10,positionText="10",positionOrder=10)] ## kevin_magnussen
hungarian[driverId==154,`:=`(grid=18,position=16,positionText="16",positionOrder=16)] ## grosjean
hungarian[driverId==841,`:=`(grid=19,position=17,positionText="17",positionOrder=17)] ## giovinazzi
hungarian[driverId==842,`:=`(grid=10,position=20,positionText="R",positionOrder=20)] ## gasly
hungarian[driverId==847,`:=`(grid=12,position=18,positionText="18",positionOrder=18)] ## russell
hungarian[driverId==849,`:=`(grid=15,position=19,positionText="19",positionOrder=19)] ## latifi
hungarian[driverId==839,`:=`(grid=14,position=14,positionText="14",positionOrder=14)] ## ocon
hungarian[driverId==844,`:=`(grid=6,position=11,positionText="11",positionOrder=11)] ## leclerc
hungarian[driverId==20,`:=`(grid=5,position=6,positionText="6",positionOrder=6)] ## vettel
### above based on Wikipedia race report; validated order of position to car number, checks out

quali_hungary<-quali[raceId==1042,.(qualifyId=.I+100000,raceId=1043,driverId,constructorId,number,q1="",q2="",q3="")]
quali_hungary<-merge(quali_hungary,hungarian[,.(driverId,position=grid)]) ## qualifying was idential for race
### again, validated order of position to car number, checks out

#############################
#### stage British Grand Prix the same way
british<-results[raceId==1042
                   ,.(raceId=1044,resultId=.I+100050,driverId,constructorId
                      ,number,grid=0,position=NA,positionText="",positionOrder=NA)]
british[driverId==1,`:=`(grid=1)] ## hamilton
british[driverId==822,`:=`(grid=2)] ## bottas
british[driverId==830,`:=`(grid=3)] ## max_verstappen
british[driverId==848,`:=`(grid=12)] ## albon
british[driverId==846,`:=`(grid=5)] ## norris
british[driverId==815,`:=`(driverId=807,number=27,grid=13)] ## nico hulkenberg drove for Perez this race
british[driverId==840,`:=`(grid=6)] ## stroll
british[driverId==817,`:=`(grid=8)] ## ricciardo
british[driverId==832,`:=`(grid=7)] ## sainz
british[driverId==826,`:=`(grid=19)] ## kvyat ## 14 + 5-place penalty
british[driverId==8,`:=`(grid=16)] ## raikkonen
british[driverId==825,`:=`(grid=14)] ## kevin_magnussen
british[driverId==154,`:=`(grid=17)] ## grosjean
british[driverId==841,`:=`(grid=15)] ## giovinazzi
british[driverId==842,`:=`(grid=11)] ## gasly
british[driverId==847,`:=`(grid=20)] ## russell ## 15 + 5-place penalty
british[driverId==849,`:=`(grid=18)] ## latifi
british[driverId==839,`:=`(grid=9)] ## ocon
british[driverId==844,`:=`(grid=4)] ## leclerc
british[driverId==20,`:=`(grid=10)] ## vettel
### above based on Wikipedia race report; validated order of position to car number, checks out

quali_british<-quali[raceId==1042,.(qualifyId=.I+100050,raceId=1044,driverId,constructorId,number,q1="",q2="",q3="")]
quali_british[driverId==815,`:=`(driverId=807,number=27)]  ## same override of Perez/Hulkenberg
quali_british<-merge(quali_british,british[,.(driverId,position=grid)])
quali_british[driverId %in% c(847,826),position:=position-5]  ## russell and kvyat qualified 5 places higher than grid
quali_british[driverId %in% c(8,825,154,841,849),position:=position+2]  ## all others moved up two
### above based on F1 website; validated order of position to car number, checks out
### https://www.formula1.com/en/results.html/2020/races/1048/great-britain/qualifying.html


### Last Step, merge these in
results<-rbind(results,hungarian,british,fill = TRUE)
quali<-rbind(quali,quali_hungary,quali_british,fill = TRUE)

### Done using code to create data ;-)

raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
<int>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1031,2020,1,70,Austrian Grand Prix,2020-07-05,13:10:00,http://en.wikipedia.org/wiki/2020_Austrian_Grand_Prix,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1032,2020,2,70,Styrian Grand Prix,2020-07-12,13:10:00,http://en.wikipedia.org/wiki/2020_Styrian_Grand_Prix,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1033,2020,3,11,Hungarian Grand Prix,2020-07-19,13:10:00,http://en.wikipedia.org/wiki/2020_Hungarian_Grand_Prix,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1034,2020,4,9,British Grand Prix,2020-08-02,13:10:00,http://en.wikipedia.org/wiki/2020_British_Grand_Prix,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1035,2020,5,9,70th Anniversary Grand Prix,2020-08-09,13:10:00,http://en.wikipedia.org/wiki/70th_Anniversary_Grand_Prix,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1036,2020,6,4,Spanish Grand Prix,2020-08-16,13:10:00,http://en.wikipedia.org/wiki/2020_Spanish_Grand_Prix,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1037,2020,7,13,Belgian Grand Prix,2020-08-30,13:10:00,http://en.wikipedia.org/wiki/2020_Belgian_Grand_Prix,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1038,2020,8,14,Italian Grand Prix,2020-09-06,13:10:00,http://en.wikipedia.org/wiki/2020_Italian_Grand_Prix,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1039,2020,9,76,Tuscan Grand Prix,2020-09-13,13:10:00,http://en.wikipedia.org/wiki/2020_Tuscan_Grand_Prix,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N


In [4]:
### merge the basic data sets together to see minimal driver/race info, and get
### qualification results and race results together
qr<-merge(
  results[,.(raceId,driverId,constructorId,grid,pos=positionOrder,positionText)]
  ,quali[,.(raceId,driverId,qPos=position)]
  ,c("raceId","driverId"),all.x=TRUE,all.y=TRUE)
qr<-merge(qr,drivers[,.(driverId,driverRef)],"driverId",all.x = TRUE)
qr<-merge(qr,races[,.(raceId,year,date,race_name=gsub(" Grand Prix","",name))]
          ,"raceId",all.x = TRUE)[order(-date,-year,raceId,pos)]

qr[grid==0,grid:=99] ## possibly unnecessary, but I first filled NAs with 0s
qr[is.na(qPos),qPos:=99]

## create a custom calculation to better account for midfield rankings
## official points are awarded only to the top 10 classified finishers
##  and this was expanded from top 6 not long ago.
## this calculation judges the top 20. Note that the grid has included 
##  more cars in the past. My focus was to try and work through basic 
##  predictions for the 2020 British GP, so I started with 20.
## This (poorly named) column will be used for most things in the future.
## This should be easier to optimize later.
## Twitter @HasRussellScore: yes, in this scoring system ;-) 
## Specifically, this just squares the result of subtracting 20 from the result
## so 1st = (20-1)^2 = 361; 10th = 100; 20th = 0 (22nd also 0)

qr[,predPts:=(20-pmin(20,pos))^2]


### finish data combination with the manually updated team merger
qr<-merge(qr,constructors[,.(constructorId,mergedId,teamRef=constructorRef)],"constructorId")

## overwrite raw qualifying position (pre-penalty) with the grid position, if we don't
##  have qualifying results (half the races or so)
qr[raceId %in% qr[,.(minQ=min(qPos)),raceId][minQ==99,raceId],qPos:=grid]


In [5]:
### ROLLING CALCULATIONS
###  need to lag first to avoid leakage, and also override to get first (N-1) included
###  chose to lag first, then do roll1-10, and plug missings with the prior rolls, 
###  cascading 10 to 1

## 1-race lag of the proxy "points" column
qr[order(driverId,raceId),lagPredPts:=shift(predPts,n=1,fill = NA),driverId]

## N-period moving averages of the proxy points
qr[order(driverRef,raceId),`:=`(
  roll2D=round(frollmean(lagPredPts,2,fill = NA,align = "right"))
  ,roll3D=round(frollmean(lagPredPts,3,fill = NA,align = "right"))
  ,roll4D=round(frollmean(lagPredPts,4,fill = NA,align = "right"))
  ,roll5D=round(frollmean(lagPredPts,5,fill = NA,align = "right"))
  ,roll6D=round(frollmean(lagPredPts,6,fill = NA,align = "right"))
  ,roll7D=round(frollmean(lagPredPts,7,fill = NA,align = "right"))
  ,roll8D=round(frollmean(lagPredPts,8,fill = NA,align = "right"))
  ,roll9D=round(frollmean(lagPredPts,9,fill = NA,align = "right"))
  ,roll10D=round(frollmean(lagPredPts,10,fill = NA,align = "right"))
  ),driverRef]

### now cascade missings
qr[is.na(lagPredPts),lagPredPts:=0]
qr[is.na(roll2D),roll2D:=lagPredPts]
qr[is.na(roll3D),roll3D:=roll2D]
qr[is.na(roll4D),roll4D:=roll3D]
qr[is.na(roll5D),roll5D:=roll4D]
qr[is.na(roll6D),roll6D:=roll5D]
qr[is.na(roll7D),roll7D:=roll6D]
qr[is.na(roll8D),roll8D:=roll7D]
qr[is.na(roll9D),roll9D:=roll8D]
qr[is.na(roll10D),roll10D:=roll9D]

### same idea for teams; attention to assist team continuity only paid for last
### two team changes. there were a lot of in/out teams 10-20 years ago.
teamPoints<-qr[,.(teamLagPredPoints=mean(lagPredPts)),.(mergedId,raceId)]
teamPoints[order(mergedId,raceId),`:=`(
  roll5T=round(frollmean(teamLagPredPoints,5,fill = NA,align = "right"))
  ,roll10T=round(frollmean(teamLagPredPoints,10,fill = NA,align = "right"))
),mergedId]

### add results back in
qr<-merge(qr,teamPoints[,.(mergedId,raceId,roll5T,roll10T)]
          ,c("mergedId","raceId"),all.x=TRUE)

### Produce ranks on the raw rolling stats
### note that the teams could have been ranked 1-10, but I chose to allow them 
### to be applied to the driver scale, so that number scale fits the others.
### this way we don't get scared to see Latifi with a 10 next to his other 
### 19s and 20s.
qr[,`:=`(
  rkRoll5D=frank(-roll5D)
  ,rkRoll10D=frank(-roll10D)
  ,rkRoll5T=frank(-roll5T)  ## team ranks will be tied with mates, but prefer 1.5-19.5 to 1-10
  ,rkRoll10T=frank(-roll10T)
  ),.(raceId)]
## end rolling points
########################

In [6]:
########################
## add basic blends
## just starting out with an extremely simple, even blend
## driver's last 5, driver's last 10, team's last 5, team's last 10, and 50/50 qualifying
## putting both qualifying would be mostly redundant, but penalties dropping a racer many
## spots doesn't indicate how fast he/she may be on race day; though it also is an indication
## that they'll have to fight through more traffic on race day

qr[,combined:=round((rkRoll5D+rkRoll10D+rkRoll5T+rkRoll10T+0.5*qPos+0.5*pmin(20,grid))/5,1)]


## for most days leading up to the race, we don't have qualifying information.
## so as I was walking my son through the basic stats, we can't use predictions
## that require qualifying results and grid position.
## So this is an even simpler blend that excludes that data. Until race Saturday, this
## is all you would be able to use to predict upcoming races.

qr[,pPreQual:=round((rkRoll5D+rkRoll10D+rkRoll5T+rkRoll10T)/4,1)]

## there is so much more to add, but this is a simple way to get started.
## the easiest thing to add would be to use the official driver and team standings
## to see how that compares to my 5/10 race rolling average, and expanded proxy points
## intuitively, this is trouble in the first few races. E.g. Verstappen was the first 
## racer out of the first 2020 GP. Should we predict he'll finish 20th in the next race?
## So my system here should be a better indicator. But without that calculated, it's just
## an unproven hunch. And, as a season settles in, the difference will not be as meaningful.
## Driver's track history might be interesting. Something to detect significant breaks in form
## such as the midseason Albon/Gasly switch in 2019 or possibly Racing Point 2020 (??)

########################

In [7]:
## Let's look at the past two British GPs to get a look at the various ranking measures
## Any ranking measure can be thought of as a prediction, and an ensembling candidate.
## It looks sane. It predicts the winner right both times, despite the polesitter earning
## second both races. Not bad, though not overly impressive either ;-)
## All in all it wasn't wrong on much, other than nearly random retirements like Verstappen
## in 2018 and Vettel in 2019.
## It would be neat to create a retirement prediction. When I was looking over the basics
## and getting started, I looked a probability of win/podium/points/retirment by grid position
## and was surprised to find P12 the most dangerous spot on the grid. Not sure why that is,
## other than a guess that 12 sees an opportunity at points within their grasp and perhaps
## tries to overachieve early. But why not 11? Perhaps just a fluke, but the trend held up
## as I stretch further and further back.
## So far, P12 and Romain Grosjean for the retirement predictor :-)
## I'll see if I can add the retirement stats back in here for EDA purposes.

qr[race_name=="British" & year==2018
            ,.(driverRef,teamRef,race_name,year,rk5D=round(rkRoll5D),rk10D=round(rkRoll10D)
               ,rk5T=floor(rkRoll5T),rk10T=floor(rkRoll10T),qPos,grid
               ,combined,pPreQual,pos
            )][order(combined)]

qr[race_name=="British" & year==2019
            ,.(driverRef,teamRef,race_name,year,rk5D=round(rkRoll5D),rk10D=round(rkRoll10D)
               ,rk5T=floor(rkRoll5T),rk10T=floor(rkRoll10T),qPos,grid
               ,combined,pPreQual,pos
            )][order(combined)]


driverRef,teamRef,race_name,year,rk5D,rk10D,rk5T,rk10T,qPos,grid,combined,pPreQual,pos
<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
vettel,ferrari,British,2018,1,1,1,1,2,2,1.4,1.2,1
raikkonen,ferrari,British,2018,5,3,1,1,3,3,2.8,2.8,3
hamilton,mercedes,British,2018,3,2,5,3,1,1,3.0,3.5,2
max_verstappen,red_bull,British,2018,2,5,3,5,5,5,4.2,4.0,15
bottas,mercedes,British,2018,6,4,5,3,4,4,4.6,4.8,4
ricciardo,red_bull,British,2018,4,6,3,5,6,6,5.0,4.8,5
kevin_magnussen,haas,British,2018,7,10,7,13,7,7,8.9,9.4,9
hulkenberg,renault,British,2018,10,7,9,7,11,11,9.0,8.5,6
sainz,renault,British,2018,8,10,9,7,16,16,10.1,8.6,17


driverRef,teamRef,race_name,year,rk5D,rk10D,rk5T,rk10T,qPos,grid,combined,pPreQual,pos
<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
hamilton,mercedes,British,2019,1,1,1,1,2,2,1.4,1.2,1
bottas,mercedes,British,2019,2,2,1,1,1,1,1.6,1.8,2
vettel,ferrari,British,2019,4,3,3,3,6,6,3.9,3.4,16
leclerc,ferrari,British,2019,5,5,3,3,3,3,4.0,4.2,3
max_verstappen,red_bull,British,2019,4,4,5,5,4,4,4.5,4.6,5
gasly,red_bull,British,2019,6,6,5,5,5,5,5.6,5.8,4
sainz,mclaren,British,2019,7,7,7,7,13,13,8.4,7.2,6
ricciardo,renault,British,2019,8,8,9,9,7,7,8.4,8.8,7
norris,mclaren,British,2019,11,10,7,7,8,8,8.9,9.1,11


In [8]:
### We can evaluate how well our predictions do compared to the other calculated methods
### one of which is the starting grid order.
### So here we calculate the MAE (mean absolute error) of each prediction method.
### My simple method doesn't win each time, but overall does edge out the other methods.
### (maeComb 3.60 vs maeGrid 3.72, rolling 10 team 3.92)
### Yes, they're all quite simple at this point, but start somewhere!
### They also might get a benefit from not being forced into a rank, so they're left in raw
### form including decimals. Forcing them to a ranking would be fairer evaluation and a 
### simpler way to read/compare the results.

expected<-qr[year>=2017 & !is.na(pos),.(
  maeComb=round(mae(pos,combined),2)
  ,maePreQual=round(mae(pos,pPreQual),2)
  ,maeGrid=round(mae(pos,pmin(20,grid)),2)
  ,maeQ=round(mae(pos,qPos),2)
  ,mae5d=round(mae(pos,rkRoll5D),2)
  ,mae10d=round(mae(pos,rkRoll10D),2)
  ,mae5t=round(mae(pos,rkRoll5T),2)
  ,mae10t=round(mae(pos,rkRoll10T),2)
  ),.(
    race_name,year)][order(-maeComb)]

## average the error across all races for all methods; lowest error is the blend
t(sapply(expected[,3:10],mean))





maeComb,maePreQual,maeGrid,maeQ,mae5d,mae10d,mae5t,mae10t
3.736815,4.031259,3.660815,3.765481,4.272,4.228296,4.299852,4.241926


In [9]:
### One nice thing about having a model is looking at differences between expected outcomes.
### For F1, the further apart our predictions are from the final standings, the more likely 
### collisions, breakdowns, or other retirements occurred. 
### Sure enough, in the last three years, the most difficult to predict were the German and
### Brazilian GPs from 2019, which makes sense.
### If you run it through 2000, the 2001 German GP is the highest, and it contained two early
### crashes, one requiring a full race restart.
### And one would suppose the 2017 Italian GP was the most uneventful race of the last few years.

### look at how the methods fared for all races; least accurate at top, most at bottom
expected

race_name,year,maeComb,maePreQual,maeGrid,maeQ,mae5d,mae10d,mae5t,mae10t
<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Emilia Romagna,2020,24.24,30.14,4.35,4.47,30.55,30.35,30.02,30.35
Turkish,2020,19.79,24.60,5.31,3.77,19.32,20.12,29.76,29.76
Hungarian,2021,7.45,7.50,7.40,7.10,7.40,7.50,7.55,7.55
German,2019,7.39,7.50,7.20,7.40,7.60,7.60,7.40,7.70
Brazilian,2019,6.73,7.04,6.00,6.50,7.10,6.90,7.30,7.05
Austrian,2018,6.05,6.20,5.60,5.60,6.00,6.20,6.15,6.70
Azerbaijan,2018,5.83,5.67,6.30,6.70,5.85,5.55,5.60,5.95
Azerbaijan,2017,5.67,5.79,5.60,9.45,5.80,5.75,6.25,5.90
Italian,2020,5.59,5.88,4.50,4.50,5.80,5.85,6.00,6.15


In [10]:
### Since this doesn't actually predict the future yet, let's still leave it as close
### as possible for those that skip to the end.

preds_british<-qr[race_name=="British" & year==2020
            ,.(driverRef,teamRef,race_name,year
               ,rk5D=round(rkRoll5D),rk10D=round(rkRoll10D)
               ,rk5T=floor(rkRoll5T),rk10T=floor(rkRoll10T)
               ,rkPredict=frank(combined+0.001*grid) ## grid as tie breaker
               ,rkPreQual=frank(pPreQual+0.001*rkRoll5D)  ## rk5D as tie breaker 
               ,qPos
               ,grid
               ,predict_preQual=pPreQual
               ,prediction=combined
               ,finish=pos
            )][order(rkPredict)]

preds_british

## As with last time, every method I used points to Hamilton winning (rk5D - predict_preQual columns)
## The interesting things to note would be differences from grid, and differences from standings 
## The standings are easy to get, but I haven't added them yet. TODO for next week.
## Grid, we see Albon predicted 5th despite his 12th qualifying.
## That is also noteworthy, as he would have been ranked 4th before qualifying. So the blended
## method that leans on grid results does make a small impact.
## Vettel predicted 8th from 10th on grid. 
## Stroll 9th despite 6th on grid. Racing Point "looks" fast again and 
##   Stroll managed 4th last race from P3 on grid, so he could hold on.
## It has Hulkenberg in at 11th, up from grid P13. I didn't check that it included his prior
##   races properly, but it probably did. However, the boost is from Racing Points's team finishes.
## As in the outset comments, nothing too unexpected here. But, it's something

## Though I don't have the easy points data included just yet, it is as follows, prior to the race
##  1, 63, Hamilton (prediction: 1)
##  2, 58, Bottas (prediction: 2)
##  3, 33, Verstappen (prediction: 3)
##  4, 26, Norris (prediction: 6)
##  5, 22, Albon (prediction: 5)
##  6, 22, Perez (prediction: --)
##  7, 18, Leclerc (prediction: 4)
##  8, 18, Stroll (prediction: 9)
##  9, 15, Sainz (prediction: 7)
## 10,  9, Vettel (prediction: 8)
## 11,  8, Ricciardo (prediction: 10)
## 12,  6, Gasly (prediction: 14)
## 13,  4, Ocon (prediction: 12)
## 14,  2, Giovinazzi (prediction: 15)
## 15,  1, Kvyat (prediction: 17)
## 16,  1, Magnussen (prediction: 16)
## 17,  0, Raikkonen (prediction: 13)
## 18,  0, Latifi (prediction: 19)
## 19,  0, Grosjean (prediction: 18)
## 20,  0, Russell (prediction: 20)


driverRef,teamRef,race_name,year,rk5D,rk10D,rk5T,rk10T,rkPredict,rkPreQual,qPos,grid,predict_preQual,prediction,finish
<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
hamilton,mercedes,British,2020,1,1,1,1,1,1,1,1,1.2,1.2,1
bottas,mercedes,British,2020,3,2,1,1,2,2,2,2,2.0,2.0,11
max_verstappen,red_bull,British,2020,2,3,3,3,3,3,3,3,3.0,3.0,2
leclerc,ferrari,British,2020,7,4,7,5,4,6,4,4,6.0,5.6,3
albon,red_bull,British,2020,6,5,3,3,5,4,12,12,4.5,6.0,8
norris,mclaren,British,2020,5,8,5,7,6,7,5,5,6.5,6.2,5
sainz,mclaren,British,2020,4,7,5,7,7,5,7,7,6.0,6.2,13
vettel,ferrari,British,2020,9,6,7,5,8,8,10,10,7.0,7.6,10
stroll,racing_point,British,2020,13,12,9,9,9,10,6,6,11.0,10.0,9


In [23]:
### export the various freatures/rankings.
fwrite(preds_british,"submission.csv")